# Decision Tree

In [1]:
from typing import List
import math

In [2]:
def entropy(class_probabilities: List[float]) -> float:
    '''given a list of class probabilities, compute the entropy'''
    return sum(-p * math.log(p, 2)
                for p in class_probabilities
                if p > 0)

In [5]:
assert entropy([0.5, 0.5]) == 1

In [6]:
from typing import Any
from collections import Counter

In [8]:
def class_probabilities(labels: List[Any]) -> List[float]:
    total_count = len(labels)
    return [count / total_count
            for count in Counter(labels).values()]

def data_entropy(labels: List[Any]) -> float:
    return entropy(class_probabilities(labels))    

In [11]:
assert data_entropy(['a']) == 0
assert data_entropy([True, False]) == 1